In [1]:
import ROOT as rt
import sys
import os
import Plotting as plot
sys.path.append(os.getcwd())
from quantileRegression import quantileRegression, setupJoblib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#%matplotlib inline
%matplotlib inline
#reload(plt)
plt.style.use('seaborn-talk')
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (8,8)
import copy as cp



Welcome to ROOTaaS 6.06/08


In [2]:
# Variable you want to correct
ylist = ["R9", "S4", "SigmaIeIe", "EtaWidth", "PhiWidth", "CovarianceIphiIphi",'CovarianceIetaIphi']#,"PhoIso03","ChIso03","ChIso03worst"]
# Input variables used for the regression
X = ['Pt', 'ScEta', 'Phi', 'rho']  # <<-- list
n_jobs=30
quantiles = [ 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9 ]

In [3]:
# Upload Dataframes:
i= str(5)
periods=[1,2,3,4,5]
qr_mc1= quantileRegression("mc")
qr_mc2= quantileRegression("mc")
qr_data= quantileRegression("data")
qr_mc1.loadDFh5("/mnt/t3nfs01/data01/shome/giulioisac/dataMC/MTR/df_mc_"+i+".h5", 0, -1)
qr_mc2.loadDFh5("/mnt/t3nfs01/data01/shome/giulioisac/dataMC/MTR/df_mc_"+i+".h5", 0, -1)
qr_data.loadDFh5("/mnt/t3nfs01/data01/shome/giulioisac/dataMC/MTR/df_data_"+i+".h5", 0, -1)
dfdata= qr_data.getDF() 

Loading dataframe from :  /mnt/t3nfs01/data01/shome/giulioisac/dataMC/MTR/df_mc_5.h5
Selecting events   [ 0 ,  2000000 ]  out of  2000000
number of events: 2000000
Loading dataframe from :  /mnt/t3nfs01/data01/shome/giulioisac/dataMC/MTR/df_mc_5.h5
Selecting events   [ 0 ,  2000000 ]  out of  2000000
number of events: 2000000
Loading dataframe from :  /mnt/t3nfs01/data01/shome/giulioisac/dataMC/MTR/df_data_5.h5
Selecting events   [ 0 ,  2000000 ]  out of  2000000
number of events: 2000000


In [4]:
qr_mc1.correctAllY(X, ylist, quantiles,EBEE="EB", relativePath= "weights_Period_"+i) 
qr_mc2.correctAllY(X, ylist, quantiles,EBEE="EE", relativePath= "weights_Period_"+i) 
dfmc1= qr_mc1.getDF() 
dfmc2= qr_mc2.getDF() 
frame2 = [dfmc1, dfmc2] 
data2 = pd.concat(frame2) 
qr_mc= quantileRegression("mc")
qr_mc.df = data2.reset_index(drop=True) 
qr_mcPhoIso0=cp.deepcopy(qr_mc)
qr_dataPhoIso0=cp.deepcopy(qr_data)
qr_mcPhoIso0.df['PhoIso03']=0
qr_dataPhoIso0.df['PhoIso03']=0
qr_mcPhoIso0.df['PhoIso03_corr']=0

Loading corrected targets from :  correctedTargetsweights_Period_5_EB.h5
Loading corrected targets from :  correctedTargetsweights_Period_5_EE.h5


In [5]:
#%time qr_mc.correctPhoIso("../phoIsoStoch/corrections_hybrid_moriond17_v2.root",n_jobs=n_jobs)
qr_mc.df["PhoIso03_corr"]=qr_mc.df["PhoIso03"]

In [6]:
showerShapes = ['R9', 'SigmaIeIe', 'PhiWidth', 'EtaWidth', 'CovarianceIetaIphi', 'S4',"CovarianceIphiIphi"]

mvas = [ ("moriondPhoID",dict(correct=[], rhoSubtraction=True) ), 
#("moriondPhoIDcorrShowerShapes",dict(correct=showerShapes, rhoSubtraction=True) )]#,
##("moriondPhoIDcorrIso",dict(correct=['PhoIso03'],rhoSubtraction=True)),
("moriondPhoIDcorrAll",dict(correct=showerShapes+['PhoIso03'], rhoSubtraction=True))]

weights = ("./phoIDMVAweights/HggPhoId_barrel_Moriond2017_wRhoRew.weights.xml","./phoIDMVAweights/HggPhoId_endcap_Moriond2017_wRhoRew.weights.xml")

In [7]:
%time qr_data.computeIdMvas( mvas[:1],  weights, n_jobs=n_jobs )
%time qr_dataPhoIso0.computeIdMvas( mvas[:1],  weights, n_jobs=n_jobs )
%time qr_mc.computeIdMvas( mvas, weights, n_jobs=n_jobs )
%time qr_mcPhoIso0.computeIdMvas( mvas, weights, n_jobs=n_jobs )
dfdataPhoIso0=  qr_dataPhoIso0.getDF()
dfmc = qr_mc.getDF()
dfmcPhoIso0 = qr_mcPhoIso0.getDF()
dfdata = qr_data.getDF()

Computing moriondPhoID, correcting {'rhoSubtraction': True, 'correct': []}
inside init
inside init
inside init
inside init
entering readerEE
entering readerEE
inside init
entering readerEE
inside init
inside init
entering readerEE
inside init


JoblibValueError: JoblibValueError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
/swshare/anaconda/lib/python2.7/runpy.py in _run_module_as_main(mod_name='ipykernel.__main__', alter_argv=1)
    169     pkg_name = mod_name.rpartition('.')[0]
    170     main_globals = sys.modules["__main__"].__dict__
    171     if alter_argv:
    172         sys.argv[0] = fname
    173     return _run_code(code, main_globals, None,
--> 174                      "__main__", fname, loader, pkg_name)
        fname = '/mnt/t3nfs01/data01/swshare/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py'
        loader = <pkgutil.ImpLoader instance>
        pkg_name = 'ipykernel'
    175 
    176 def run_module(mod_name, init_globals=None,
    177                run_name=None, alter_sys=False):
    178     """Execute a module's code without importing it

......................

In [ ]:
dfmc["moriondPhoIDIso0"]=dfmcPhoIso0["moriondPhoID"]
dfmc["moriondPhoIDIso0corrAll"]=dfmcPhoIso0["moriondPhoIDcorrAll"]

In [ ]:
dfEB   = dfdata  .query('(-1.444 < ScEta and ScEta <1.444) and (mass > 70 and mass < 110)  and (ChIso03 < 5)') 
dfEE   = dfdata  .query('(ScEta < -1.57 or ScEta >1.57)  and (mass > 70 and mass < 110)  and (ChIso03 < 5) and (R9>0.8)')
dfIsoEB   = dfdataPhoIso0  .query('(-1.444 < ScEta and ScEta <1.444) and (mass > 70 and mass < 110)  and (ChIso03 < 5)') 
dfIsoEE   = dfdataPhoIso0  .query('(ScEta < -1.57 or ScEta >1.57)  and (mass > 70 and mass < 110)  and (ChIso03 < 5) and (R9>0.8)')
dfmcEB   = dfmc  .query('(-1.444 < ScEta and ScEta <1.444)  and (mass > 70 and mass < 110) and (ChIso03 < 5)') 
dfmcEE   = dfmc  .query('(ScEta < -1.57 or ScEta >1.57) and (mass > 70 and mass < 110)  and (ChIso03 < 5) and (R9>0.8)')
dfmcIsoEB   = dfmc  .query('(-1.444 < ScEta and ScEta <1.444)  and (mass > 70 and mass < 110) and (ChIso03 < 5)') 
dfmcIsoEE   = dfmc  .query('(ScEta < -1.57 or ScEta >1.57) and (mass > 70 and mass < 110)  and (ChIso03 < 5) and (R9>0.8)')

In [ ]:
bins = np.linspace(-1,1,101)
var = 'moriondPhoID'
mclabels = [ '', 'corrAll', 'Iso0', 'Iso0corrAll']
mcstyles = [ dict(label='MC no corr',color='green'), 
             dict(label='MC corr ',color='darkturquoise'), 
             dict(label='MC no Corr iso=0',color='orange'),
             dict(label='MC corr iso=0',color='blue'), 
           ]

for dfs in [ (dfEB,dfmcEB,dfIsoEB), (dfEE,dfmcEE,dfIsoEE) ]: 
    data1, mc = plot.data_mc(dfs[0],dfs[1],var,bins,weight="weight",mclabels=mclabels)
    data2, mc = plot.data_mc(dfs[2],dfs[1],var,bins,weight="weight",mclabels=mclabels)
    draw_datas_mc(bins, data1, data2, zip(mc,mcstyles), var='ID MVA', markersize=6, alpha=1, linewidth=2.5, ratio=False, figsize=(10,8))
  

In [ ]:
def draw_datas_mc(bins,data1,data2, mc,figsize=(8,6),var=None,logy=False,ratio=False,
                **kwargs):
    
    mcstyle=dict(alpha=0.5,linewidth=0)
    mcstyle.update(kwargs)
    datastyle=cp.copy(kwargs)
    datastyle["alpha"] = 1. # No transparency for data
    
    binw=bins[1]-bins[0]
    if ratio:
        fig, axes = plt.subplots(2,figsize=figsize,sharex=True,gridspec_kw = {'height_ratios':[3, 1]})
        top = axes[0]
        bottom = axes[1]
        fig.tight_layout()
    else:
        fig = plt.figure(figsize=figsize)
        axes = None
        top = plt
    
    # FIXME: assumes uniform binning
    xc = bins[1:]-binw*0.5
    
    #print mc
    for hist, style in mc:
        pkwargs = cp.copy(mcstyle)
        pkwargs.update(style)
        top.plot(xc+binw*0.5,hist,**pkwargs)
    top.errorbar( xc+binw*0.5, data1,ls='None', xerr=np.ones_like(data1)*binw*0.5, yerr=np.sqrt(data1), color='black', 
                 label='Data', fmt='o', **datastyle )
    top.errorbar( xc+binw*0.5, data2,ls='None', xerr=np.ones_like(data2)*binw*0.5, yerr=np.sqrt(data2), color='red', 
                 label='Data iso=0', fmt='o', **datastyle )
        
    if axes == None: axes = fig.axes
    
    if ratio:
        ratios = []
        for hist, style in mc:
            rdata = data / hist
            rdata_err = np.sqrt(data) / hist
            ratios.append((rdata,rdata_err))
            rkwargs = {}
            if len(mc) == 1: rkwargs['color'] = 'black'
            elif "color" in style: rkwargs['color'] = style['color']
            rkwargs.update(datastyle)
            bottom.errorbar( xc+binw*0.5, rdata,ls='None', xerr=np.ones_like(rdata)*binw*0.5, yerr=rdata_err, 
                        **rkwargs)
        
        bottom.plot( (bins[0],bins[-1]), (1,1), 'k-' )
        bottom.set_ylabel('Data / MC')
        bottom.set_ylim(0,2)
    
    if logy:
        axes[0].set_yscale('log')
    axes[0].set_xlim(bins[0],bins[-1])
    
    unit = None    
    if var != None:
        if type(var) != str:
            var, unit = var
        if unit: var += " (%s)" % unit
        axes[-1].set_xlabel(var)
    ylabel = 'Events / %1.3g' % binw
    if unit:
        ylabel += ' %s' % unit
    axes[0].set_ylabel(ylabel)

    top.legend(loc='best')
        

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rc('xtick', labelsize=18) 
mpl.rc('ytick', labelsize=18) 
mpl.rc('lines', linewidth=2)
mpl.rc('axes', linewidth=2)

dm=dfmcIsoEB
dd=dfIsoEB
sec="EB"
ylist=["R9", "S4", "SigmaIeIe", "EtaWidth", "PhiWidth", "CovarianceIphiIphi",'CovarianceIetaIphi','Pt', 'ScEta', 'Phi', 'rho']
boxes=15

for yvar in ylist:
    print "plotting ", yvar
    if yvar == "R9":
        if sec=="EB":
            rv  = np.linspace(0,1.0,boxes)
        else:
            rv  = np.linspace(0.75,1.0,boxes)
    if yvar == "S4":
        if sec=="EB":
            rv = np.linspace(0,1.0,boxes)
        else:
            rv = np.linspace(0.35,1.0,boxes)
    if yvar == "SigmaIeIe":
        if sec=="EB":
            rv = np.linspace(0.004,0.014,boxes)
        else:
            rv = np.linspace(0.014,0.020,boxes)
    if yvar == "EtaWidth":
        if sec=="EB":
            rv = np.linspace(0.0,0.05,boxes)
        else:
            rv = np.linspace(0.0,0.05,boxes)
    if yvar == "PhiWidth":
        if sec=="EB":
            rv = np.linspace(0.0,0.2,boxes)
        else:
            rv = np.linspace(0.0,0.2,boxes)
    if yvar == "CovarianceIphiIphi":
        if sec=="EB":
            rv  = np.linspace(0.0, 0.02,boxes)
        else:
            rv  = np.linspace(0.0, 0.02,boxes)
    if yvar == "CovarianceIetaIphi":
        if sec=="EB":
            rv = np.linspace(-0.0002, 0.0002,boxes) 
        else:
            rv = np.linspace(-0.0002, 0.0002,boxes) 
    if yvar=="ScEta":
        rv = np.linspace(-2.5, 2.5 ,boxes)
    if yvar=="Pt":
        rv = np.linspace(20, 80 ,boxes)
    if yvar=='rho':
        rv = np.linspace(0, 40 ,boxes)
    if yvar=="Phi":
        rv = np.linspace(-3.14, 3.14 ,boxes)
    y="moriondPhoID"
    y1="moriondPhoIDcorrAll"
    fig = plt.figure(figsize=(18,9))
    ax = fig.add_subplot(111)
    ax.set_xlabel(yvar)
    dm['bin'+yvar] = pd.cut(dm[yvar], rv)
    bpmc = dm.boxplot(column=y,by='bin'+yvar, return_type='dict', ax=ax)
    [[item.set_color('g') for item in bpmc[key]['medians']] for key in bpmc.keys()]
    [[item.set_color('g') for item in bpmc[key]['boxes']] for key in bpmc.keys()]
    [[item.set_color('g') for item in bpmc[key]['whiskers']] for key in bpmc.keys()]
    [[item.set_color('g') for item in bpmc[key]['caps']] for key in bpmc.keys()]
    dd['bin'+yvar] = pd.cut(dd[yvar], rv)
    bpdata = dd.boxplot(column=y,by='bin'+yvar, return_type='dict', ax=ax)
    [[item.set_color('r') for item in bpdata[key]['medians']] for key in bpdata.keys()]
    [[item.set_color('r') for item in bpdata[key]['boxes']] for key in bpdata.keys()]
    [[item.set_color('r') for item in bpdata[key]['whiskers']] for key in bpdata.keys()]
    [[item.set_color('r') for item in bpdata[key]['caps']] for key in bpdata.keys()]
    plt.show()

 
    fig = plt.figure(figsize=(18,9))
    ax = fig.add_subplot(111)
    bpmc = dm.boxplot(column=y1,by='bin'+yvar, return_type='dict', ax=ax)
    [[item.set_color('b') for item in bpmc[key]['medians']] for key in bpmc.keys()]
    [[item.set_color('b') for item in bpmc[key]['boxes']] for key in bpmc.keys()]
    [[item.set_color('b') for item in bpmc[key]['whiskers']] for key in bpmc.keys()]
    [[item.set_color('b') for item in bpmc[key]['caps']] for key in bpmc.keys()]
    dd['bin'+yvar] = pd.cut(dd[yvar], rv)
    bpdata = dd.boxplot(column=y,by='bin'+yvar, return_type='dict', ax=ax)
    [[item.set_color('r') for item in bpdata[key]['medians']] for key in bpdata.keys()]
    [[item.set_color('r') for item in bpdata[key]['boxes']] for key in bpdata.keys()]
    [[item.set_color('r') for item in bpdata[key]['whiskers']] for key in bpdata.keys()]
    [[item.set_color('r') for item in bpdata[key]['caps']] for key in bpdata.keys()]
    plt.show()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rc('xtick', labelsize=18) 
mpl.rc('ytick', labelsize=18) 
mpl.rc('lines', linewidth=2)
mpl.rc('axes', linewidth=2)

dm=dfmcIsoEE
dd=dfIsoEE
sec="EE"
ylist=["R9", "S4", "SigmaIeIe", "EtaWidth", "PhiWidth", "CovarianceIphiIphi",'CovarianceIetaIphi','Pt', 'ScEta', 'Phi', 'rho']
boxes=15

for yvar in ylist:
    print "plotting ", yvar
    if yvar == "R9":
        if sec=="EB":
            rv  = np.linspace(0.75,1.0,boxes)
        else:
            rv  = np.linspace(0.75,1.0,boxes)
    if yvar == "S4":
        if sec=="EB":
            rv = np.linspace(0.35,1.0,boxes)
        else:
            rv = np.linspace(0.35,1.0,boxes)
    if yvar == "SigmaIeIe":
        if sec=="EB":
            rv = np.linspace(0.004,0.014,boxes)
        else:
            rv = np.linspace(0.014,0.020,boxes)
    if yvar == "EtaWidth":
        if sec=="EB":
            rv = np.linspace(0.0,0.05,boxes)
        else:
            rv = np.linspace(0.0,0.05,boxes)
    if yvar == "PhiWidth":
        if sec=="EB":
            rv = np.linspace(0.0,0.2,boxes)
        else:
            rv = np.linspace(0.0,0.2,boxes)
    if yvar == "CovarianceIphiIphi":
        if sec=="EB":
            rv  = np.linspace(0.0, 0.02,boxes)
        else:
            rv  = np.linspace(0.0, 0.02,boxes)
    if yvar == "CovarianceIetaIphi":
        if sec=="EB":
            rv = np.linspace(-0.0002, 0.0002,boxes) 
        else:
            rv = np.linspace(-0.0002, 0.0002,boxes) 
    if yvar=="ScEta":
        rv = np.linspace(-2.5, 2.5 ,boxes)
    if yvar=="Pt":
        rv = np.linspace(20, 80 ,boxes)
    if yvar=='rho':
        rv = np.linspace(0, 40 ,boxes)
    if yvar=="Phi":
        rv = np.linspace(-3.14, 3.14 ,boxes)
    y="moriondPhoID"
    y1="moriondPhoIDcorrAll"
    fig = plt.figure(figsize=(18,9))
    ax = fig.add_subplot(111)
    ax.set_xlabel(yvar)
    dm['bin'+yvar] = pd.cut(dm[yvar], rv)
    bpmc = dm.boxplot(column=y,by='bin'+yvar, return_type='dict', ax=ax)
    [[item.set_color('g') for item in bpmc[key]['medians']] for key in bpmc.keys()]
    [[item.set_color('g') for item in bpmc[key]['boxes']] for key in bpmc.keys()]
    [[item.set_color('g') for item in bpmc[key]['whiskers']] for key in bpmc.keys()]
    [[item.set_color('g') for item in bpmc[key]['caps']] for key in bpmc.keys()]
    dd['bin'+yvar] = pd.cut(dd[yvar], rv)
    bpdata = dd.boxplot(column=y,by='bin'+yvar, return_type='dict', ax=ax)
    [[item.set_color('r') for item in bpdata[key]['medians']] for key in bpdata.keys()]
    [[item.set_color('r') for item in bpdata[key]['boxes']] for key in bpdata.keys()]
    [[item.set_color('r') for item in bpdata[key]['whiskers']] for key in bpdata.keys()]
    [[item.set_color('r') for item in bpdata[key]['caps']] for key in bpdata.keys()]
    plt.show()

 
    fig = plt.figure(figsize=(18,9))
    ax = fig.add_subplot(111)
    bpmc = dm.boxplot(column=y1,by='bin'+yvar, return_type='dict', ax=ax)
    [[item.set_color('b') for item in bpmc[key]['medians']] for key in bpmc.keys()]
    [[item.set_color('b') for item in bpmc[key]['boxes']] for key in bpmc.keys()]
    [[item.set_color('b') for item in bpmc[key]['whiskers']] for key in bpmc.keys()]
    [[item.set_color('b') for item in bpmc[key]['caps']] for key in bpmc.keys()]
    dd['bin'+yvar] = pd.cut(dd[yvar], rv)
    bpdata = dd.boxplot(column=y,by='bin'+yvar, return_type='dict', ax=ax)
    [[item.set_color('r') for item in bpdata[key]['medians']] for key in bpdata.keys()]
    [[item.set_color('r') for item in bpdata[key]['boxes']] for key in bpdata.keys()]
    [[item.set_color('r') for item in bpdata[key]['whiskers']] for key in bpdata.keys()]
    [[item.set_color('r') for item in bpdata[key]['caps']] for key in bpdata.keys()]
    plt.show()